In [ ]:
%%capture
!pip install evaluate
!pip install sacrebleu
!pip install evaluate
!pip install transformers
!pip install SentencePiece
!pip install torch
!pip install accelerate -U

In [ ]:
import numpy as np
import pandas as pd
import evaluate
from datasets import load_dataset
from transformers import M2M100Tokenizer, M2M100ForConditionalGeneration
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

## Datasets

In [ ]:
news_en = load_dataset("news_commentary", "en-ru")
news_en = news_en["train"].train_test_split(test_size=0.2)

news_fr = load_dataset("news_commentary", "fr-ru")
news_fr = news_fr["train"].train_test_split(test_size=0.2)

news_ar = load_dataset("news_commentary", "ar-ru")
news_ar = news_ar["train"].train_test_split(test_size=0.2)

## Preprocessing and tokenization

In [ ]:
def preprocess_function(examples, target_lang, source_lang):
    prefix = f"translate {source_lang} to {target_lang}: "
    tokenizer = M2M100Tokenizer.from_pretrained(
                                            "facebook/m2m100_1.2B",
                                            src_lang=source_lang,
                                            tgt_lang=target_lang)

    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=512, truncation=True)
    return model_inputs


In [ ]:
tokenized_news_en = [preprocess_function(news, 'ru', 'en') for news in news_en]
tokenized_news_fr = [preprocess_function(news, 'ru', 'fr') for news in news_en]
tokenized_news_ar = [preprocess_function(news, 'ru', 'ar') for news in news_en]

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

metric = evaluate.load("sacrebleu")

## Model initialization

In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained('facebook/m2m100_1.2B')
tokenizer = M2M100Tokenizer.from_pretrained('facebook/m2m100_1.2B')
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
)



## Model fine-tunning on different languages

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_news_en["train"],
    eval_dataset=tokenized_news_en["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_news_fr["train"],
    eval_dataset=tokenized_news_fr["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_news_ar["train"],
    eval_dataset=tokenized_news_ar["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.save_model('/home/ubuntu/result/')